# PyTorch Interface Demo

In this notebook, we took an example from PyTorch's examples page to demonstrated how - by simply replacing the imports with syft's pytorch interface - you can run PyTorch code using OpenMined as a backend.

In [13]:
import argparse
import gym, time
import numpy as np
from itertools import count
from syft.controller import tensors, models


import syft
import syft.interfaces.torch.actual_torch as actual_torch
import syft.interfaces.torch as torch
import syft.interfaces.torch.nn as nn
import syft.interfaces.torch.nn.functional as F
import syft.interfaces.torch.optim as optim
from syft.interfaces.torch.autograd import Variable
from syft.interfaces.torch.distributions import Categorical

# import torch as torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.distributions import Categorical

gamma = 0.9
seed = 543
render = False
log_interval = 1

env = gym.make('CartPole-v0')
env.seed(seed)
torch.manual_seed(seed)

class Policy(nn.Module):
    
    def __init__(self):
        super(Policy, self).__init__()
        self.affine = nn.Linear(4, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        action_scores = self.affine(x)
        return F.softmax(action_scores, dim=1)
    

policy = Policy()

# I can't seed our sampler to have the exact same random seed as PyTorch
# so this is the next best thing (for the purposes of testing only)
# cached_actions are the random samples that pytorch chooses
cached_actions = [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, ]
actions = list()

def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(Variable(state))
    m = Categorical(probs)
    action = m.sample()
    
    # testing purposes only - COMMENT THIS OUT WHEN USING REGULAR TORCH - it's just so that 
    # the random seeds are the same for sampling as well - so that results are exactly the same as pytorch
    action = Variable(torch.IntTensor([cached_actions[len(actions)]]))
    
    actions.append(action)
    policy.saved_log_probs.append(m.log_prob(action))
    return action.data[0]

optimizer = optim.SGD(policy.parameters(), lr=0.15)

def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float32).eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

start = time.time()
running_reward = 10
for i_episode in range(4):
    state = env.reset()
    for t in range(10000):  # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _ = env.step(action)
        if render:
            env.render()
        policy.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode()
    if i_episode % log_interval == 0:
        print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
            i_episode, t, running_reward))
    if running_reward > env.spec.reward_threshold:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!".format(running_reward, t))
        break
end = time.time()
print("Four iterations took " + str(end - start) + " seconds")

# Syft Generated Output - should be identical

[2017-12-31 18:36:14,773] Making new env: CartPole-v0


Episode 0	Last length:    16	Average length: 10.06
Episode 1	Last length:     9	Average length: 10.05
Episode 2	Last length:    30	Average length: 10.25
Episode 3	Last length:    30	Average length: 10.45
Four iterations took 30.922693252563477 seconds


In [8]:
# PyTorch Generated Output

Episode 1	Last length:    16	Average length: 10.06
Episode 2	Last length:     9	Average length: 10.05
Episode 3	Last length:    30	Average length: 10.25
Episode 4	Last length:    30	Average length: 10.45
Episode 5	Last length:    43	Average length: 10.77
Episode 6	Last length:    36	Average length: 11.02
Episode 7	Last length:    84	Average length: 11.75
Episode 8	Last length:    50	Average length: 12.14
Episode 9	Last length:    21	Average length: 12.23
Episode 10	Last length:    43	Average length: 12.53
Episode 11	Last length:    32	Average length: 12.73
Episode 12	Last length:    19	Average length: 12.79
Episode 13	Last length:    28	Average length: 12.94
Episode 14	Last length:    57	Average length: 13.38
Episode 15	Last length:   126	Average length: 14.51
Episode 16	Last length:    42	Average length: 14.78
Episode 17	Last length:    98	Average length: 15.62
Episode 18	Last length:    72	Average length: 16.18
Episode 19	Last length:    98	Average length: 17.00
Episode 20	Last lengt

Episode 159	Last length:   182	Average length: 102.49
Episode 160	Last length:   199	Average length: 103.46
Episode 161	Last length:   165	Average length: 104.07
Episode 162	Last length:   188	Average length: 104.91
Episode 163	Last length:    83	Average length: 104.69
Episode 164	Last length:   152	Average length: 105.17
Episode 165	Last length:    93	Average length: 105.04
Episode 166	Last length:    97	Average length: 104.96
Episode 167	Last length:   156	Average length: 105.47
Episode 168	Last length:    75	Average length: 105.17
Episode 169	Last length:    30	Average length: 104.42
Episode 170	Last length:    85	Average length: 104.22
Episode 171	Last length:    85	Average length: 104.03
Episode 172	Last length:    75	Average length: 103.74
Episode 173	Last length:    82	Average length: 103.52
Episode 174	Last length:    87	Average length: 103.36
Episode 175	Last length:    93	Average length: 103.25
Episode 176	Last length:    83	Average length: 103.05
Episode 177	Last length:    

Episode 318	Last length:   199	Average length: 78.97
Episode 319	Last length:   199	Average length: 80.17
Episode 320	Last length:   199	Average length: 81.36
Episode 321	Last length:   199	Average length: 82.54
Episode 322	Last length:   199	Average length: 83.70
Episode 323	Last length:   191	Average length: 84.77
Episode 324	Last length:   199	Average length: 85.92
Episode 325	Last length:   158	Average length: 86.64
Episode 326	Last length:   190	Average length: 87.67


KeyboardInterrupt: 